In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import class_weight
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
import pickle
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model
from keras import optimizers, regularizers, initializers
from keras.callbacks import ModelCheckpoint, Callback
from keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
NCA1 = 50
NCA2 = 50
DROPRATE = 0.2
EP = 500
BATCH_SIZE = 256
VAL_RATIO = 0.1
TEST_RATIO = 0.1

## Loading Drugbank dataset

In [ ]:
db_moa_df= pd.read_csv('../input/dbslim_descriptors/db_moa_df.csv')
print(db_moa_df.shape)
db_moa_df.head()

In [ ]:
db_moa_df.columns

In [ ]:
db_moa_df.drop(['Unnamed: 0', 'type', 'drugbank_id', 'name', 'description',
               'pharmacodynamics', 'mechanism-of-action', 'toxicity', 'metabolism',
               'absorption', 'half-life', 'protein-binding', 'route-of-elimination',
               'clearance', 'groups', 'targets', 'enzymes', 'atc_codes', 'categories',
               'inchi', 'inchikey', 'SMILES', 'logS', 'logP', 'Water_Solubility',
               'Melting_Point', 'Boiling_Point', 'Hydrophobicity', 'Isoelectric_Point',
               'caco2_Permeability', 'pKa', 'Radioactivity', 'aliases', 'smiles'], axis=1, inplace=True)
print(db_moa_df.shape)
db_moa_df.head()

## Loading molecular descriptors

Descriptors dataframe contains 1625 molecular descriptors (including 3D descriptors) generated on the NCI database using Mordred python module.

Further Reading:
* https://en.wikipedia.org/wiki/Molecular_descriptor
* https://github.com/mordred-descriptor/mordred

In [ ]:
db_moa_descriptors_df= pd.read_csv('../input/dbslim_descriptors/dbslim_descriptors_df.csv',low_memory=False)
print(db_moa_descriptors_df.shape)
db_moa_descriptors_df.head()

In [ ]:
# function to coerce all data types to numeric

def coerce_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')

In [ ]:
coerce_to_numeric(db_moa_descriptors_df, db_moa_descriptors_df.columns)
db_moa_descriptors_df.head()

In [ ]:
db_moa_descriptors_df = db_moa_descriptors_df.fillna(0)
db_moa_descriptors_df.head()

## Scaling and Principal component analysis (PCA) 

In [ ]:
db_moa_scaler1 = StandardScaler()
db_moa_scaler1.fit(db_moa_descriptors_df.values)
db_moa_descriptors_df = pd.DataFrame(db_moa_scaler1.transform(db_moa_descriptors_df.values),
                                     columns=db_moa_descriptors_df.columns)

In [ ]:
nca = NCA1
cn = ['col'+str(x) for x in range(nca)]

In [ ]:
db_moa_transformer1 = KernelPCA(n_components=nca, kernel='rbf', n_jobs=-1)
db_moa_transformer1.fit(db_moa_descriptors_df.values)
db_moa_descriptors_df = pd.DataFrame(db_moa_transformer1.transform(db_moa_descriptors_df.values),
                                     columns=cn)
print(db_moa_descriptors_df.shape)
db_moa_descriptors_df.head()

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(db_moa_descriptors_df.values,
                                                              db_moa_df.values, 
                                                              test_size=TEST_RATIO)

In [ ]:
X_train, y_train, X_valid, y_valid = iterative_train_test_split(X_train, y_train, 
                                                                test_size=VAL_RATIO)

In [ ]:
def Find_Optimal_threshold(target, predicted):
    
    rng = np.arange(0.0, 0.99, 0.001)
    f1s = np.zeros((rng.shape[0],predicted.shape[1]))
    for i in range(0,predicted.shape[1]):
        for j,t in enumerate(rng):
            p = np.array((predicted[:,i])>t, dtype=np.int8)
            scoref1 = f1_score(target[:,i], p, average='binary')
            f1s[j,i] = scoref1
            
    threshold = np.empty(predicted.shape[1])
    for i in range(predicted.shape[1]):
        threshold[i] = rng[int(np.where(f1s[:,i] == np.max(f1s[:,i]))[0][0])]
        
    return threshold

## Sklearn SVC Model

In [ ]:
parameters = {'estimator__class_weight':['balanced'],
              'estimator__kernel':['rbf','sigmoid'], 
              'estimator__C':[1,0.5,0.25], 'estimator__gamma':['auto','scale']}
db_moa_svc = GridSearchCV(OneVsRestClassifier(SVC(probability=True,
                                                  random_state=23)), 
                          parameters, cv=3, scoring='roc_auc',n_jobs=-1)

In [ ]:
result = db_moa_svc.fit(X_train, y_train)

In [ ]:
pred = db_moa_svc.predict_proba(X_valid)
pred_svc_t = np.copy(pred)
roc_auc_score(y_valid,pred)

In [ ]:
threshold = Find_Optimal_threshold(y_valid, pred)
print(threshold)

In [ ]:
pred = db_moa_svc.predict(X_test)
f1_score(y_test,pred,average='macro')

In [ ]:
pred = db_moa_svc.predict_proba(X_test)
pred_svc = np.copy(pred)
svc_auc_score = roc_auc_score(y_test,pred)
print(svc_auc_score)

In [ ]:
pred[pred<=threshold] = 0
pred[pred>threshold] = 1
svc_f1_score = f1_score(y_test,pred,average='macro')
print(svc_f1_score)
print(f1_score(y_test,pred,average=None))

## Keras Neural Network Model

In [ ]:
db_moa_model = Sequential()
db_moa_model.add(Dense(128, input_dim=db_moa_descriptors_df.shape[1], 
                       kernel_initializer='he_uniform'))
db_moa_model.add(BatchNormalization())
db_moa_model.add(Activation('tanh'))
db_moa_model.add(Dropout(rate=DROPRATE))
db_moa_model.add(Dense(64,kernel_initializer='he_uniform'))
db_moa_model.add(BatchNormalization())
db_moa_model.add(Activation('tanh'))
db_moa_model.add(Dropout(rate=DROPRATE))
db_moa_model.add(Dense(32,kernel_initializer='he_uniform'))
db_moa_model.add(BatchNormalization())
db_moa_model.add(Activation('tanh'))
db_moa_model.add(Dropout(rate=DROPRATE))
db_moa_model.add(Dense(db_moa_df.shape[1],kernel_initializer='he_uniform',activation='sigmoid'))

In [ ]:
db_moa_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('db_moa_model.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

In [ ]:
hist = db_moa_model.fit(X_train, y_train, 
                        validation_data=(X_valid,y_valid),epochs=EP, batch_size=BATCH_SIZE, 
                        callbacks=[checkpoint])

In [ ]:
plt.ylim(0., 1.)
plt.plot(hist.epoch, hist.history["loss"], label="Train loss")
plt.plot(hist.epoch, hist.history["val_loss"], label="Valid loss")

In [ ]:
db_moa_model.load_weights('db_moa_model.h5')

In [ ]:
pred = db_moa_model.predict(X_valid)
pred_nn_t = np.copy(pred)

In [ ]:
threshold = Find_Optimal_threshold(y_valid, pred)
print(threshold)

In [ ]:
pred = db_moa_model.predict(X_test)
pred_nn = np.copy(pred)
nn_auc_score = roc_auc_score(y_test,pred)
print(nn_auc_score)

In [ ]:
pred[pred<=threshold] = 0
pred[pred>threshold] = 1
nn_f1_score = f1_score(y_test,pred,average='macro')
print(nn_f1_score)
print(f1_score(y_test,pred,average=None))

## Gradient Boosting of Keras Model with SVC

In [ ]:
inp = db_moa_model.input
out = db_moa_model.layers[-2].output
db_moa_model_gb = Model(inp, out)

In [ ]:
X_train = db_moa_model_gb.predict(X_train)
X_test = db_moa_model_gb.predict(X_test)
X_valid = db_moa_model_gb.predict(X_valid)

In [ ]:
data = np.concatenate((X_train,X_test,X_valid),axis=0)

In [ ]:
db_moa_scaler2 = StandardScaler()
db_moa_scaler2.fit(data)
X_train = db_moa_scaler2.transform(X_train)
X_test = db_moa_scaler2.transform(X_test)
X_valid = db_moa_scaler2.transform(X_valid)

In [ ]:
data = np.concatenate((X_train,X_test,X_valid),axis=0)

In [ ]:
nca = NCA2

In [ ]:
db_moa_transformer2 = KernelPCA(n_components=nca, kernel='rbf', n_jobs=-1)
db_moa_transformer2.fit(data)
X_train = db_moa_transformer2.transform(X_train)
X_test = db_moa_transformer2.transform(X_test)
X_valid = db_moa_transformer2.transform(X_valid)

In [ ]:
nca = X_train.shape[1]
parameters = {'estimator__class_weight':['balanced'],
              'estimator__kernel':['rbf','sigmoid'], 
              'estimator__C':[1,0.5,0.25], 'estimator__gamma':['scale','auto']}

db_moa_svc_gb = GridSearchCV(OneVsRestClassifier(SVC(probability=True,
                                                     random_state=23)), 
                             parameters, cv=3, scoring='roc_auc',n_jobs=-1)

In [ ]:
result = db_moa_svc_gb.fit(X_train, y_train)

In [ ]:
pred = db_moa_svc_gb.predict_proba(X_valid)
pred_svc_gb_t = np.copy(pred)
roc_auc_score(y_valid,pred)

In [ ]:
threshold = Find_Optimal_threshold(y_valid, pred)
print(threshold)

In [ ]:
pred = db_moa_svc_gb.predict(X_test)
f1_score(y_test,pred,average='macro')

In [ ]:
pred = db_moa_svc_gb.predict_proba(X_test)
pred_svc_gb = np.copy(pred)
svc_gb_auc_score = roc_auc_score(y_test,pred)
print(svc_gb_auc_score)

In [ ]:
pred[pred<=threshold] = 0
pred[pred>threshold] = 1
svc_gb_f1_score = f1_score(y_test,pred,average='macro')
print(svc_gb_f1_score)
print(f1_score(y_test,pred,average=None))

## Gradient Boosting of Keras Model with XGBoost

In [ ]:
parameters = {'estimator__learning_rate':[0.05,0.1,0.15],'estimator__n_estimators':[75,100,125], 'estimator__max_depth':[3,5,7],
              'estimator__booster':['gbtree','dart'],'estimator__reg_alpha':[0.1,0.05],'estimator__reg_lambda':[0.5,1.]}

db_moa_xgb_gb = GridSearchCV(OneVsRestClassifier(XGBClassifier(random_state=32)), parameters, cv=3, scoring='roc_auc',n_jobs=-1)

In [ ]:
result = db_moa_xgb_gb.fit(X_train, y_train)

In [ ]:
pred = db_moa_xgb_gb.predict_proba(X_valid)
pred_xgb_gb_t = np.copy(pred)
roc_auc_score(y_valid,pred)

In [ ]:
threshold = Find_Optimal_threshold(y_valid, pred)
print(threshold)

In [ ]:
pred = db_moa_xgb_gb.predict(X_test)
f1_score(y_test,pred,average='macro')

In [ ]:
f1_score(y_test,pred,average=None)

In [ ]:
pred = db_moa_xgb_gb.predict_proba(X_test)
pred_xgb_gb = np.copy(pred)
xgb_gb_auc_score = roc_auc_score(y_test,pred)
print(xgb_gb_auc_score)

In [ ]:
pred[pred<=threshold] = 0
pred[pred>threshold] = 1
xgb_gb_f1_score = f1_score(y_test,pred,average='macro')
print(xgb_gb_f1_score)
print(f1_score(y_test,pred,average=None))

In [ ]:
pred = (pred_svc_t+pred_nn_t+pred_svc_gb_t+pred_xgb_gb_t)/4.

In [ ]:
threshold = Find_Optimal_threshold(y_valid, pred)
print(threshold)

In [ ]:
pred = (pred_svc+pred_nn+pred_svc_gb+pred_xgb_gb)/4.
ave_auc_score = roc_auc_score(y_test,pred)
print(ave_auc_score)
pred[pred<=threshold] = 0
pred[pred>threshold] = 1
ave_f1_score = f1_score(y_test,pred,average='macro')
print(ave_f1_score)
print(f1_score(y_test,pred,average=None))

## Saving models, transformer and scaler

In [ ]:
with open('db_moa_svc.pkl', 'wb') as fid:
    pickle.dump(db_moa_svc, fid)
with open('db_moa_transformer1.pkl', 'wb') as fid:
    pickle.dump(db_moa_transformer1, fid)
with open('db_moa_transformer2.pkl', 'wb') as fid:
    pickle.dump(db_moa_transformer2, fid)
with open('db_moa_scaler1.pkl', 'wb') as fid:
    pickle.dump(db_moa_scaler1, fid)
with open('db_moa_scaler2.pkl', 'wb') as fid:
    pickle.dump(db_moa_scaler2, fid)
with open('db_moa_svc_gb.pkl', 'wb') as fid:
    pickle.dump(db_moa_svc_gb, fid)
with open('db_moa_xgb_gb.pkl', 'wb') as fid:
    pickle.dump(db_moa_xgb_gb, fid)

## For loading saved model

```python
with open('db_moa_svc.pkl', 'rb') as fid:
    db_moa_svc = pickle.load(fid)
 ```

## F1 Score

In [ ]:
sns.set(style="whitegrid")
ax = sns.barplot(x=[svc_f1_score,nn_f1_score,svc_gb_f1_score,xgb_gb_f1_score,ave_f1_score],
                 y=['SVC','NN','SVC_GB','XGB_GB','ave'])
ax.set(xlim=(0.05, None))

## ROC-AUC

In [ ]:
sns.set(style="whitegrid")
ax = sns.barplot(x=[svc_auc_score,nn_auc_score,svc_gb_auc_score,xgb_gb_auc_score,ave_auc_score],
                 y=['SVC','NN','SVC_GB','XGB_GB','ave'])
ax.set(xlim=(0.30, None))